## Elasticsearch: The Definitive Guide - Python

Following the examples in the book, here are Python snippets that achieve the same effect.

Documentation for the Python libs:

Low-level API:

https://elasticsearch-py.readthedocs.io/en/master/index.html

Expressive DSL API (more "Pythonic")

http://elasticsearch-dsl.readthedocs.io/en/latest/index.html

Github repo for DSL API:

https://github.com/elastic/elasticsearch-dsl-py


In [1]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [2]:
import index
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from pprint import pprint

es = Elasticsearch(
    'localhost',
    # sniff before doing anything
    sniff_on_start=True,
    # refresh nodes after a node fails to respond
    sniff_on_connection_fail=True,
    # and also every 60 seconds
    sniffer_timeout=60
)

r = index.populate()
print('{} items created'.format(len(r['items'])))

# Let's repopulate the index as we deleted 'gb' in earlier chapters:
# Run the script: populate.ipynb

14 items created


### Normalizing Tokens

Breaking text into tokens is only half the job. To make those tokens more easily searchable, they need to go through a normalization process to remove insignificant differences between otherwise identical words, such as uppercase versus lowercase. Perhaps we also need to remove significant differences, to make esta, ésta, and está all searchable as the same word. Would you search for déjà vu, or just for deja vu?

This is the job of the token filters, which receive a stream of tokens from the tokenizer. You can have multiple token filters, each doing its particular job. Each receives the new token stream as output by the token filter before it.

#### In That Case

The most frequently used token filter is the lowercase filter, which does exactly what you would expect; it transforms each token into its lowercase form:

```
GET /_analyze?tokenizer=standard&filters=lowercase
The QUICK Brown FOX! 
```

In [7]:
text = 'The QUICK Brown FOX!'# contains some uppercase words
analyzed_text = [x['token'] for x in es.indices.analyze\
                 (tokenizer='standard', filter=['lowercase'], text=text)['tokens']]
print(','.join(analyzed_text))

the,quick,brown,fox


To make this automatic as part of the analysis process, we can create a custom analyzer:
```
PUT /my_index
{
  "settings": {
    "analysis": {
      "analyzer": {
        "my_lowercaser": {
          "tokenizer": "standard",
          "filter":  [ "lowercase" ]
        }
      }
    }
  }
}
```

In [15]:
# first delete the index from previous chapters, if it exists
if es.indices.exists('my_index'): 
    es.indices.delete('my_index')

In [16]:
#es.indices.create('my_index')
from elasticsearch_dsl import analyzer, Index
my_custom_analyzer = analyzer('my_lowercaser',
        tokenizer='standard',
        filter='lowercase')
i = Index('my_index')
i.analyzer(my_custom_analyzer)

In [17]:
es.indices.analyze(index='my_index', analyzer='my_lowercaser', text=text)

{'tokens': [{'end_offset': 3,
   'position': 0,
   'start_offset': 0,
   'token': 'the',
   'type': '<ALPHANUM>'},
  {'end_offset': 9,
   'position': 1,
   'start_offset': 4,
   'token': 'quick',
   'type': '<ALPHANUM>'},
  {'end_offset': 15,
   'position': 2,
   'start_offset': 10,
   'token': 'brown',
   'type': '<ALPHANUM>'},
  {'end_offset': 19,
   'position': 3,
   'start_offset': 16,
   'token': 'fox',
   'type': '<ALPHANUM>'}]}